In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm



In [ ]:
text = 'Mình nặng 70.5kg, nặng 175.8 cm.'
digit = '\d+'

import re
print(re.findall(digit, text))

In [ ]:
path = "../input/bms-molecular-translation/"
test = pd.read_csv(path + 'sample_submission.csv', index_col=0)

labels_path = path + "train_labels.csv"
df_train_alls = pd.read_csv(labels_path)
df_train_labels=df_train_alls[:]

In [ ]:
fully_qualified_path = path + "train/{}/{}/{}/{}.png"
convert_image_id_to_path = lambda image_id_details :fully_qualified_path.format(image_id_details[0], image_id_details[1], image_id_details[2], image_id_details) 

In [ ]:
s_type=[inchi.split("/")[1] for inchi in tqdm(df_train_alls["InChI"])]
print(set(s_type))

In [ ]:
df_train_labels['image_path']=df_train_labels['image_id'].apply(convert_image_id_to_path)

In [ ]:
df_train_labels.tail()

In [ ]:
elements=['Cl', 'C','Br','B','Si','S' 'H', 'P', 'O', 'N', 'I', 'F']
elements_revise=['G', 'C','R','B','L','S' 'H', 'P', 'O', 'N', 'I', 'F']

df_train_labels['formula'] = df_train_labels['InChI'].apply(lambda x: x.split('/')[1]) 

In [ ]:
def revise_element(element):
    revise=element
    if "Cl" in element:
        revise=revise.replace("Cl", "G")
    if "Br" in element:
        revise=revise.replace("Br", "R")
    if "Si" in element:
        revise=revise.replace("Si", "L")
    return revise;

In [ ]:
df_train_labels['formula']=df_train_labels['formula'].apply(revise_element)

In [ ]:
def number_of_carbon(element):
    nums={"0","1","2","3","4","5","6","7","8","9"}
    result=""
    for i in range(1,len(element)):
        if element[i] in nums:
            result=result+element[i]
        else:
            break    
    return int(result)

In [ ]:
df_train_labels['C']=df_train_labels['formula'].apply(number_of_carbon)

In [ ]:
df_train_labels.head()

In [ ]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.applications import DenseNet121
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.densenet import preprocess_input
from keras.models import Model

In [ ]:
def create_X(path):
    filename = path
    image = load_img(filename, target_size=(128, 128))
         # convert the image pixels to a numpy array
    image = img_to_array(image)
    image=255-image[:,:,0]
    return image.tolist()

In [ ]:
df_train_labels['X']=df_train_labels['image_path'].apply(create_X)

In [ ]:
df_train_labels.head()

In [ ]:
X=np.array(df_train_labels.X.values.tolist())
X.shape


In [ ]:
Y=df_train_labels["C"].values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

In [ ]:
X=X.reshape(X.shape[0],X.shape[1],X.shape[2],1)


In [ ]:
m=X.shape[0]
test_val=int(m/5)
X_val, Y_val = X[:m-2*test_val:m-test_val], Y[:m-2*test_val:m-test_val]
X_train, Y_train = X[:m-2*test_val], Y[:m-2*test_val]
X_test,Y_test=X[m-test_val:], Y[m-test_val:]

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = Sequential()
 
# Thêm Convolutional layer với 32 kernel, kích thước kernel 3*3
# dùng hàm sigmoid làm activation và chỉ rõ input_shape cho layer đầu tiên
model.add(Conv2D(1, (3, 3), activation='sigmoid', input_shape=(128,128,1)))
model.add(Conv2D(1, (3, 3), activation='sigmoid'))
# Thêm Max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten layer chuyển từ tensor sang vector
model.add(Flatten())

# Thêm Fully Connected layer với 128 nodes và dùng hàm sigmoid
model.add(Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(0.001)))
model.add(Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(0.001)))
model.add(Dense(16, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(0.001)))

# Output layer với 10 node và dùng softmax function để chuyển sang xác xuất.
model.add(Dense(1, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(0.01)))
# 6. Compile model, chỉ rõ hàm loss_function nào được sử dụng, phương thức 
# đùng để tối ưu hàm loss function.
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
H = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
          batch_size=32, epochs=10, verbose=1)

In [ ]:
H.history

In [ ]:
# 8. Vẽ đồ thị loss, accuracy của traning set và validation set
fig = plt.figure()
numOfEpoch = 3
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), 1-np.array(H.history['accuracy']), label='accuracy')
plt.plot(np.arange(0, numOfEpoch), 1-np.array(H.history['val_accuracy']), label='validation accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

In [ ]:
for i in range(100):
    y_predict = model.predict(X_test[i].reshape(1,128,128,1))
    print("Dự đoan {}vs thức tế {}".format(y_predict[0][0],Y_test[i]))

In [ ]:
def convert_image_id_2_path(image_id: str) -> str:
    return path + "test/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

In [ ]:
def visualize_image(image_id, label):
    plt.figure(figsize=(10, 8))
    
    image = cv2.imread(convert_image_id_2_path(image_id))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.title(f"{label}", fontsize=14)
    plt.axis("off")
    
    plt.show()

In [ ]:
def visualize_image_denoise(image_id):
    plt.figure(figsize=(10, 8))  
    image = cv2.imread(convert_image_id_2_path(image_id), cv2.IMREAD_GRAYSCALE)
    _, blackAndWhite = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 2:   #filter small dotted regions
            img2[labels == i + 1] = 255
    image = cv2.bitwise_not(img2)
    plt.imshow(image)    
    plt.axis("off")
    plt.show()

In [ ]:
def visualize_train_batch(image_ids, labels):
    plt.figure(figsize=(32, 24))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(convert_image_id_to_path(image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"{label[:50]}...", fontsize=10)
        plt.axis("off")
    
    plt.show()

In [ ]:
i=0
visualize_image(test.index[i], test.index[i])
visualize_image_denoise(test.index[i])

In [ ]:
tmp_df = df_train_labels[:1]
image_ids = tmp_df['image_id']
labels = tmp_df["InChI"].values
visualize_train_batch(image_ids, labels)
print(labels)


In [ ]:
tmp_df = df_train_labels[7:9]
image_ids = tmp_df['image_id']
labels = tmp_df["InChI"].values
visualize_train_batch(image_ids, labels)


In [ ]:
print('Length of training-data:',len(df_train_labels))
print('Number of unique chemical identifier:',len(df_train_labels['InChI'].value_counts().index))
print('Max count of any chemical identifier in trainign data:',max(df_train_labels['InChI'].value_counts().values))

In [ ]:
h_shape=[]
w_shape=[]
aspect_ratio=[]
for idx,image_id in enumerate(df_train_labels.image_id.values[:3000]):
    image = cv2.imread(df_train_labels['image_path'][idx])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h_shape.append(image.shape[0])
    w_shape.append(image.shape[1])
    aspect_ratio.append(1.0 * (image.shape[1] / image.shape[0]))

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplots_adjust(top = 0.5, bottom=0.01, hspace=1, wspace=0.4)
plt.subplot(2, 2, 1)
plt.hist(np.array(h_shape) * np.array(w_shape), bins=50)
plt.xticks(rotation=45)
plt.title("Area Image Distribution", fontsize=14)
plt.subplot(2, 2, 2)
plt.hist(h_shape, bins=50)
plt.title("Height Image Distribution", fontsize=14)
print()
plt.subplot(2, 2, 3)
plt.hist(w_shape, bins=50)
plt.title("Width Image Distribution", fontsize=14)
plt.subplot(2, 2, 4)
plt.hist(aspect_ratio, bins=50)
plt.title("Aspect Ratio Distribution", fontsize=14);

In [ ]:
plt.scatter(h_shape,w_shape)
plt.show()

In [ ]:
def display_odd(h,w):
    for i in range(len(h_shape)):
        if h_shape[i]>h or w_shape[i]>w:
            visualize_train_batch(df_train_labels.loc[i:i].image_id, df_train_labels.loc[i:i].InChI)


In [ ]:
display_odd(800,1500)

In [ ]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.applications import DenseNet121
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.densenet import preprocess_input
from keras.models import Model

In [ ]:
# extract features from each image
def extract_features():
    
 # load the model
    model = DenseNet121()
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # summarize
    #print(model.summary())
 # extract features from each image
    features = dict()
    for idx,name in enumerate(df_train_labels['image_path'].values[:100]):
        filename = name
        image = load_img(filename, target_size=(224, 224))
         # convert the image pixels to a numpy array
        image = img_to_array(image)
         # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
         # prepare the image for the DenseNet121 model
        image = preprocess_input(image)
         # get features
        feature = model.predict(image, verbose=0)
         # store feature
        features[df_train_labels['image_id'][idx]] = feature
        #print('>%s' % name)
    return features

In [ ]:
features = extract_features()
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))


In [ ]:
# extract texts for images
def load_text():
    mapping = dict()
    for idx,text in enumerate(df_train_labels['InChI'].values[:101]):
        mapping[df_train_labels['image_id'][idx]]=text
    return mapping

In [ ]:
def to_vocabulary(descriptions):
    all_desc = set()
    for key,value in descriptions.items():
        all_desc.update([value])
    return all_desc

In [ ]:
texts = load_text()
vocabulary  = to_vocabulary(texts)

In [ ]:
print('Loaded: %d ' % len(texts))
print('Vocabulary Size: %d' % len(vocabulary))

In [ ]:
from tqdm.auto import tqdm
import Levenshtein

In [ ]:
tqdm.pandas()

In [ ]:
test = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
train=df_train_labels
train['InChI_list'] = train['InChI'].progress_apply(lambda x: x.split('/'))
train['InChI_length'] = train['InChI_list'].progress_apply(len)
InChI_df = train['InChI_list'].progress_apply(pd.Series)
train = pd.concat([train, InChI_df.add_prefix('InChI_')], axis=1)

In [ ]:
display(train)

In [ ]:
def get_score(y_true, y_pred):
    scores = []
    for true, pred in zip(y_true, y_pred):
        score = Levenshtein.distance(true, pred)
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score

In [ ]:
mode_concat_string = ''
for i in range(11):
    mode_string = train[f'InChI_{i}'].fillna('nan').mode()[0]
    if mode_string != 'nan':
        if i == 0:
            mode_concat_string += mode_string
        else:
            mode_concat_string += '/' + mode_string

In [ ]:
print(mode_concat_string)

In [ ]:
y_true = train['InChI'].values
y_pred = [mode_concat_string] * len(train)
score = get_score(y_true, y_pred)

In [ ]:
print(score)

In [ ]:
test['InChI'] = mode_concat_string
output_cols = ['image_id', 'InChI']
display(test[output_cols])
test[output_cols].to_csv('submission.csv', index=False)